## Pairs trading for Tech sector fund(XLK)

In [53]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import yfinance as yf
from scipy.spatial.distance import pdist,squareform
from scipy.cluster.hierarchy import linkage,fcluster
from statsmodels.tsa.stattools import coint
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')

Load the data.

We are taking many companies from the tech sector that are listed(in the list), so around ~69

* Also previous 10 years data

In [54]:
ticker = ["NVDA","AAPL","MSFT","AVGO","PLTR","AMD","CSCO","IBM","MU","ORCL","CRM","LRCX","AMAT","APP","QCOM","INTU","ACN","KLAC","TXN","NOW","INTC","APH","ADBE","ADI","ANET","PANW","CRWD","CDNS","SNPS","GLW","TEL","ADSK","MSI","STX","WDC","NXPI","FTNT","ROP","WDAY","DDOG","MPWR","FICO","CTSH","DELL","MCHP","KEYS","HPE","TER","FSLR","TDY","JBL","ON","NTAP","HPQ","PTC","VRSN","TYL","TRMB","IT","CDW","GDDY","SNDK","GEN","FFIV","SMCI","ZBRA","AKAM","EPAM","SWKS"]

In [55]:
df=yf.download(tickers=ticker,start='2015-01-01',auto_adjust=False,timeout=30)

[*********************100%***********************]  69 of 69 completed


In [56]:
df.head()

Price       Adj Close                                                         \
Ticker           AAPL        ACN       ADBE        ADI       ADSK       AKAM   
Date                                                                           
2015-01-02  24.237553  74.252632  72.339996  44.354500  59.529999  63.250000   
2015-01-05  23.554741  72.998909  71.980003  43.547901  58.660000  61.939999   
2015-01-06  23.556959  72.472366  70.529999  42.525677  57.500000  60.669998   
2015-01-07  23.887280  73.993515  71.110001  42.972919  57.380001  60.160000   
2015-01-08  24.805080  75.121880  72.919998  43.731583  58.799999  61.570000   

Price                                             ...    Volume           \
Ticker           AMAT   AMD      ANET        APH  ...       TDY      TEL   
Date                                              ...                      
2015-01-02  21.764570  2.67  3.917500  12.086211  ...  145700.0   948900   
2015-01-05  21.066992  2.66  3.987500  11.865398  ...  266200.0   953400   
2015-01-06  20.273499  2.63  3.988125  11.714433  ...  243000.0  1448800   
2015-01-07  20.465324  2.58  4.071250  11.914968  ...  245300.0  1929000   
2015-01-08  21.119314  2.61  4.132500  12.147049  ...  240300.0  1635300   

Price                                                                         \
Ticker          TER     TRMB      TXN       TYL       VRSN     WDAY      WDC   
Date                                                                           
2015-01-02  1030300  1106000  4020100  216200.0   684500.0  1630000  1278547   
2015-01-05  1877000  1254900  5599300  270300.0   874400.0  1591400  2199752   
2015-01-06  2377700  2291600  5468400  239600.0  1289300.0  1584000  2665713   
2015-01-07   902400  2096400  4577100  220300.0   552400.0  1106100  2054222   
2015-01-08  1626800  2036000  6295200  264700.0  1282600.0  2090100  3351291   

Price                 
Ticker          ZBRA  
Date                  
2015-01-02  411800.0  
2015-01-05  420300.0  
2015-01-06  527500.0  
2015-01-07  467800.0  
2015-01-08  324400.0  

[5 rows x 414 columns]

In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2759 entries, 2015-01-02 to 2025-12-19
Columns: 414 entries, ('Adj Close', 'AAPL') to ('Volume', 'ZBRA')
dtypes: float64(364), int64(50)
memory usage: 8.7 MB


In [58]:
prices=df['Adj Close']

In [59]:
prices.head()

Ticker,AAPL,ACN,ADBE,ADI,ADSK,AKAM,AMAT,AMD,ANET,APH,...,TDY,TEL,TER,TRMB,TXN,TYL,VRSN,WDAY,WDC,ZBRA
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,24.237553,74.252632,72.339996,44.354500,59.529999,63.250000,21.764570,2.67,3.917500,12.086211,...,102.160004,51.678234,18.303957,26.910000,39.557953,107.849998,56.695358,80.410004,69.842255,77.430000
2015-01-05,23.554741,72.998909,71.980003,43.547901,58.660000,61.939999,21.066992,2.66,3.987500,11.865398,...,99.720001,50.671982,17.960175,26.030001,38.944031,105.970001,55.753574,80.010002,68.918861,76.339996
2015-01-06,23.556959,72.472366,70.529999,42.525677,57.500000,60.669998,20.273499,2.63,3.988125,11.714433,...,98.019997,50.058414,17.411997,25.660000,38.300507,104.910004,54.702747,79.419998,67.381981,75.790001
2015-01-07,23.887280,73.993515,71.110001,42.972919,57.380001,60.160000,20.465324,2.58,4.071250,11.914968,...,98.790001,51.563705,17.551361,25.340000,38.988403,106.029999,55.158768,79.349998,68.393906,77.720001
2015-01-08,24.805080,75.121880,72.919998,43.731583,58.799999,61.570000,21.119314,2.61,4.132500,12.147049,...,100.440002,52.602650,18.099550,25.670000,39.624531,109.199997,56.388046,82.779999,69.810616,79.379997


In [60]:
prices.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2759 entries, 2015-01-02 to 2025-12-19
Data columns (total 69 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2759 non-null   float64
 1   ACN     2758 non-null   float64
 2   ADBE    2759 non-null   float64
 3   ADI     2759 non-null   float64
 4   ADSK    2758 non-null   float64
 5   AKAM    2759 non-null   float64
 6   AMAT    2759 non-null   float64
 7   AMD     2759 non-null   float64
 8   ANET    2759 non-null   float64
 9   APH     2759 non-null   float64
 10  APP     1178 non-null   float64
 11  AVGO    2759 non-null   float64
 12  CDNS    2758 non-null   float64
 13  CDW     2759 non-null   float64
 14  CRM     2759 non-null   float64
 15  CRWD    1642 non-null   float64
 16  CSCO    2759 non-null   float64
 17  CTSH    2759 non-null   float64
 18  DDOG    1573 non-null   float64
 19  DELL    2350 non-null   float64
 20  EPAM    2759 non-null   float64
 21  FFIV    2759 non-nu

Dropping stocks with more than 400 missing entries & all null stocks

In [61]:
prices.drop(columns=['APP','PLTR','SNDK','CRWD','DDOG','VRSN','STX','PTC','MU','KLAC','KEYS','DELL'],inplace=True)

In [62]:
prices.dropna(inplace=True)

In [63]:
prices.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2558 entries, 2015-10-19 to 2025-12-19
Data columns (total 57 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2558 non-null   float64
 1   ACN     2558 non-null   float64
 2   ADBE    2558 non-null   float64
 3   ADI     2558 non-null   float64
 4   ADSK    2558 non-null   float64
 5   AKAM    2558 non-null   float64
 6   AMAT    2558 non-null   float64
 7   AMD     2558 non-null   float64
 8   ANET    2558 non-null   float64
 9   APH     2558 non-null   float64
 10  AVGO    2558 non-null   float64
 11  CDNS    2558 non-null   float64
 12  CDW     2558 non-null   float64
 13  CRM     2558 non-null   float64
 14  CSCO    2558 non-null   float64
 15  CTSH    2558 non-null   float64
 16  EPAM    2558 non-null   float64
 17  FFIV    2558 non-null   float64
 18  FICO    2558 non-null   float64
 19  FSLR    2558 non-null   float64
 20  FTNT    2558 non-null   float64
 21  GDDY    2558 non-nu

#### Setting Parameters

In [64]:
kclusters=8
alpha_coint=0.05
half_life=(5,60)
zenter=2.0
zexit=0.5

#### Picking Pairs

First we pick pairs with clustering by cluster analysis and Hierarchial clustering

* Log prices

In [65]:
log_prices=np.log(prices)

* Train, test, split

In [66]:
log_train=log_prices[:2046]
log_test=log_prices[2046:]

* Normalize (z-score)

In [67]:
train_nor_prices=(log_train-log_train.mean())/np.std(log_train)

* Distance Matrix (correlation)

In [68]:
corr=train_nor_prices.corr()
dist=1-corr
dist_vec=squareform(dist.values)

* Agglomerative Clustering

In [69]:
link=linkage(dist_vec,method='ward')

clusters=fcluster(link,kclusters,criterion='maxclust')

cluster_map=pd.Series(clusters,index=log_train.columns)

Now we see candidate pairs

In [70]:
pairs=[]
for c in np.unique(clusters):
    members=cluster_map[cluster_map==c].index
    for i in range(len(members)):
        for j in range(i+1,len(members)):
            pairs.append((members[i],members[j]))

* Cointegration filter (Engle-Granger test)

In [71]:
coint_pairs=[]
for a,b in pairs:
    score,pvalue,_=coint(log_train[a],log_train[b])
    if pvalue<alpha_coint:
        coint_pairs.append((a,b))

* hedge ratio balancing (OLS, FROZEN)

In [72]:
def hedge_ratio(y,x):
    x=sm.add_constant(x)
    model=sm.OLS(y,x).fit()
    return model.params[1]

pairs_info=[]

for a,b in coint_pairs:
    beta=hedge_ratio(log_train[a],log_train[b])
    spread=log_train[a]-beta*log_train[b]

* Half-life filter

In [75]:
spread_lag=spread.shift(1).dropna()
delta=spread.diff().dropna()

reg=sm.OLS(delta,sm.add_constant(spread_lag)).fit()
halflife=-np.log(2)/reg.params.iloc[1]

if half_life[0]<=halflife<=half_life[1]:
    pairs_info.append({
        'A':a,
        'B':b,
        'beta':beta,
        'halflife':int(round(halflife))
    })

pairs_df=pd.DataFrame(pairs_info)

In [76]:
pairs_df.head()

,A,B,beta,halflife
0,CTSH,FFIV,0.51123,49


Define trading function

In [77]:
def trade_pair(pair, data):
    A, B = pair["A"], pair["B"]
    beta = pair["beta"]
    window = pair["halflife"]

    spread = data[A] - beta * data[B]
    mean = spread.rolling(window).mean()
    std  = spread.rolling(window).std()
    z = (spread - mean) / std

    pos = np.where(z > zenter, -1,
          np.where(z < -zenter, 1, 0))

    pos = pd.Series(pos, index=data.index).ffill().fillna(0)
    pos[z.abs() < zexit] = 0

    ret = data[A].diff() - beta * data[B].diff()
    pnl = (pos.shift(1) * ret).fillna(0)

    return pnl.cumsum()

### Test

In [78]:
results = {}

for _, pair in pairs_df.iterrows():
    pnl = trade_pair(pair, log_test)
    results[f"{pair.A}-{pair.B}"] = pnl.iloc[-1]

results = pd.Series(results).sort_values(ascending=False)

print(results)

CTSH-FFIV   -0.110077
dtype: float64


#### Lets say that we invest $1000 initially

In [82]:
cash_pnl = 1000 * (np.exp(results) - 1)
cash_pnl

CTSH-FFIV   -104.235153
dtype: float64

This shows that we are loosing our money around $104.24, the pair is bleeding.